In [35]:
with open('day16_input.txt') as file:
    puzzle_input = file.read()

flow_rates = {}
tunnels = {}
valves = {}

for line in puzzle_input.splitlines():
    words = line.split()
    valve = words[1]
    valves[valve] = len(valves)
    flow_rates[valve] = int(words[4][5:-1])
    connections = line.split(', ')
    connections[0] = connections[0][-2:]
    tunnels[valve] = connections
    
visited_states = {}

def calc_max_pressure(curr, opened, time_remaining):
    if time_remaining == 0:
        return 0
    
    if (curr, opened, time_remaining) in visited_states:
        return visited_states[(curr, opened, time_remaining)]

    pressures = []
    
    if flow_rates[curr] > 0 and not opened[valves[curr]]:
        next_opened = list(opened)
        
        next_opened[valves[curr]] = True
        pressures.append(flow_rates[curr] * (time_remaining - 1) +
                         calc_max_pressure(curr, tuple(next_opened), time_remaining - 1))
        
    for valve in tunnels[curr]:
        #print('going to ', valve)
        pressures.append(calc_max_pressure(valve, opened, time_remaining - 1))
    
    max_pressure = max(pressures)
    visited_states[(curr, opened, time_remaining)] = max_pressure
    return max_pressure

print(calc_max_pressure('AA', tuple(False for valve in valves), 30))

1862


In [ ]:
with open('day16_input.txt') as file:
    puzzle_input = file.read()

flow_rates = {}
tunnels = {}
valves = {}

for line in puzzle_input.splitlines():
    words = line.split()
    valve = words[1]
    valves[valve] = len(valves)
    flow_rates[valve] = int(words[4][5:-1])
    connections = line.split(', ')
    connections[0] = connections[0][-2:]
    tunnels[valve] = connections
    
visited_states = {}

def calc_max_pressure(curr, curr2, opened, time_remaining):
    if time_remaining == 0:
        return 0
    
    if (curr, curr2, opened, time_remaining) in visited_states:
        return visited_states[(curr, curr2, opened, time_remaining)]
    
    pressures = []
    next_opened = list(opened)
    
    if flow_rates[curr] > 0 and not opened[valves[curr]]:
        next_opened[valves[curr]] = True
        
        if flow_rates[curr2] > 0 and not opened[valves[curr2]]:
            next_opened[valves[curr2]] = True

            pressures.append(flow_rates[curr] * (time_remaining - 1) + flow_rates[curr2] * (time_remaining - 1) +
                             calc_max_pressure(curr, curr2, tuple(next_opened), time_remaining - 1))
            
        for valve2 in tunnels[curr2]:
            pressures.append(flow_rates[curr] * (time_remaining - 1) +
                             calc_max_pressure(curr, valve2, opened, time_remaining - 1))
        
    for valve in tunnels[curr]:
        if flow_rates[curr2] > 0 and not opened[valves[curr2]]:
            next_opened[valves[curr2]] = True

            pressures.append(flow_rates[curr2] * (time_remaining - 1) +
                             calc_max_pressure(valve, curr2, tuple(next_opened), time_remaining - 1))
            
        for valve2 in tunnels[curr2]:
            pressures.append(calc_max_pressure(valve, valve2, opened, time_remaining - 1))
    
    max_pressure = max(pressures)
    visited_states[(curr, curr2, opened, time_remaining)] = max_pressure
        
    return max_pressure
        
print(calc_max_pressure('AA', 'AA', tuple(False for valve in valves), 26))


In [94]:
import re
import time
import itertools


def read_puzzle(file):
  with open(file) as f:
    return [re.findall('[A-Z]+|\d+', line[1:]) for line in f.readlines()]


def solve(puzzle):
  graph = {valve:leads for valve, _, *leads in puzzle}
  flows = {valve: int(flow) for valve, flow, *_ in puzzle if flow != '0'}
  indicies = {valve: 1 << i for i, valve in enumerate(flows)}
  distances = {(v,l): 1 if l in graph[v] else 1000 for l in graph for v in graph}

  # floyd-warshall = Distance for any possible pair of valves
  for k, i, j in itertools.permutations(graph, 3):
    distances[i, j] = min(distances[i, j], distances[i, k] + distances[k, j])


  def visit(valve, minutes, bitmask, pressure, answer):
    answer[bitmask] = max(answer.get(bitmask, 0), pressure)
    for valve2, flow in flows.items():
      remaining_minutes = minutes - distances[valve, valve2] - 1
      if indicies[valve2] & bitmask or remaining_minutes <= 0: continue
      visit(valve2, remaining_minutes, bitmask|indicies[valve2], pressure + flow * remaining_minutes, answer)
    return answer

  
  part1     = max(visit('AA', 30, 0, 0, {}).values())
  visited2  = visit('AA', 26, 0, 0, {})
  print('a', visited2)
  part2     = max(v1+v2 for bitm1, v1 in visited2.items()
                  for bitm2, v2 in visited2.items() if not bitm1 & bitm2)
  
  return part1, part2


time_start = time.perf_counter()
print(solve(read_puzzle('day16_input.txt')))
print(time.perf_counter()-time_start)


a {0: 0, 1: 399, 3: 511, 8195: 555, 16387: 525, 5: 529, 7: 577, 13: 563, 21: 697, 29: 714, 149: 738, 1045: 1001, 4117: 868, 37: 799, 69: 784, 71: 816, 85: 928, 101: 1034, 197: 829, 213: 907, 453: 845, 1221: 1037, 2245: 856, 16581: 841, 325: 804, 581: 834, 1093: 984, 1109: 1084, 2117: 820, 8261: 820, 16453: 799, 24645: 823, 133: 609, 135: 641, 4245: 748, 165: 829, 229: 1054, 389: 629, 2437: 638, 8581: 641, 16773: 632, 645: 713, 1157: 876, 1173: 1043, 5253: 772, 2181: 645, 2693: 850, 4229: 651, 8325: 645, 16517: 624, 16519: 640, 18565: 633, 24709: 648, 261: 553, 2309: 578, 8453: 577, 16645: 559, 517: 688, 1029: 810, 1037: 726, 5141: 922, 1413: 884, 3205: 885, 17541: 882, 1285: 822, 3077: 828, 5125: 814, 9221: 822, 17413: 819, 2053: 574, 2565: 830, 10245: 586, 18437: 577, 4101: 644, 4109: 661, 8197: 577, 24581: 583, 16389: 547, 16391: 579, 9: 484, 4105: 649, 8201: 604, 17: 591, 25: 625, 145: 649, 1041: 912, 1169: 964, 4113: 819, 4121: 723, 33: 719, 35: 751, 53: 847, 1189: 1036, 293: 803, 

(1862, 2422)
0.7282650839988491
